In [1]:
import gpn.model
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import torch
from transformers import AutoModel, AutoModelForMaskedLM, AutoTokenizer
import h5py
import sys
from tqdm import tqdm
import numpy as np
sys.path.append('../')
import utils
import os
from tqdm import tqdm
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# model = AutoModel.from_pretrained("/home/ztang/multitask_RNA/model/GPN_finetune/checkpoint-2000000").to('cuda')
# tokenizer = AutoTokenizer.from_pretrained("/home/ztang/multitask_RNA/model/GPN_finetune/checkpoint-2000000")
# model = AutoModel.from_pretrained('songlab/gpn-brassicales').to('cuda')
# tokenizer = AutoTokenizer.from_pretrained('songlab/gpn-brassicales')
 
model = AutoModel.from_pretrained("/home/ztang/multitask_RNA/model/GPN_human/checkpoint-2000000").to('cuda')
tokenizer = AutoTokenizer.from_pretrained("/home/ztang/multitask_RNA/model/GPN_human/checkpoint-2000000")
model.eval();




celltype = 'HepG2'

file = h5py.File('/home/ztang/multitask_RNA/data/lenti_MPRA/HepG2_data.h5','r')

/home/ztang/.conda/envs/gpn_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at /home/ztang/multitask_RNA/model/GPN_human/checkpoint-2000000 were not used when initializing ConvNetModel: ['cls.decoder.3.weight', 'cls.decoder.0.bias', 'cls.decoder.2.bias', 'cls.decoder.2.weight', 'cls.decoder.3.bias', 'cls.decoder.0.weight']
- This IS expected if you are initializing ConvNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ConvNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassificatio

In [2]:
gpn_output = h5py.File('../../data/lenti_MPRA_embed/gpn_'+celltype+'.h5','w')
batch_size = 32
output_cache = []
for i in tqdm(range(0,len(file['seq']),batch_size)):

    seq = file['seq'][i:i+batch_size].astype('U230')
    input_ids = tokenizer(seq.tolist(), return_tensors="pt", return_attention_mask=False, return_token_type_ids=False)["input_ids"]
    with torch.no_grad():
        output_seq = model(input_ids.to('cuda')).last_hidden_state.cpu().detach().numpy()
    output_cache.extend(output_seq)
gpn_output.create_dataset(name='seq',data = np.array(output_cache))
gpn_output.create_dataset(name='mean',data = file['mean'][:])


  0%|          | 0/4372 [00:00<?, ?it/s]/home/ztang/.conda/envs/gpn_env/lib/python3.9/site-packages/torch/nn/modules/conv.py:309: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv1d(input, weight, bias, self.stride,
100%|██████████| 4372/4372 [02:21<00:00, 30.96it/s]


<HDF5 dataset "mean": shape (139877,), type "<f8">

In [3]:
gpn_output.keys()

<KeysViewHDF5 ['mean', 'seq']>

In [4]:
gpn_output.close()

In [5]:
len(file['seq'])

139877

In [14]:
len(seq)

5

In [8]:
gpn_output.close()